In [1]:
import persistence_homology as ph

In [2]:
import csv

In [3]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output

In [4]:
folder_path = './lung_segmentations/Lung segmentation-10'
edges_path = '/edges.csv'
vertices_path = '/vertices.csv'

In [5]:
vertices = read_csv(folder_path + vertices_path)
edges = read_csv(folder_path + edges_path)

In [15]:
directions = [[0,0,1], [0,1,0], [1,0,0]]

# Filtration Algorithm Start

In [ ]:
for direction in directions:
    bz = ph.BettiZero(direction, vertices, edges)
    components, mergers, verts, births = bz.compute_persistence()
    intervals = ph.compute_intervals(births, mergers)
    print(intervals)
    print(ph.compute_largest_bar(intervals))